In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import re
import nltk
import math
import pickle

import xgboost

from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.linear_model import Perceptron

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from Bio.SeqUtils.ProtParam import ProteinAnalysis
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, cross_val_predict

%matplotlib inline

In [2]:
# read in data from csv file
df = pd.read_csv('proteins-5-functions.csv')

df.head()

,classification,name,sequence,structureId,type
0,TRANSFERASE,NaN,MEIYEGKLTAEGLRFGIVASRFNHALVDRLVEGAIDCIVRHGGREE...,5MPP,4
1,HYDROLASE,NaN,MKFTLTIAGLLAVGSTAAPTTEKRNPGGIDYVQNYNGDVADFQYNE...,3M4F,0
2,TRANSFERASE,NaN,MRGSHHHHHHGSMKRAVITGLGIVSSIGNNQQEVLASLREGRSGIT...,2BYY,4
3,HYDROLASE,NaN,STGSATTTPIDSLDDAYITPVQIGTPAQTLNLDFDTGSSDLWVFSS...,4YCY,0
4,TRANSFERASE,NaN,GSGMMRYLHKIELELNRLTSRYPFFKKIAFDAEIIKLVDDLNVDEN...,3AQC,4


In [3]:
# read in more immune system sequences data from csv file
df_2 = pd.read_csv('uniprot-human-immune.csv')

df_2.head()

,name,sequence,classification,type
0,tr|Q6FGW4|Q6FGW4_HUMAN,MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRD...,IMMUNE SYSTEM,1
1,tr|A0A024R5Z3|A0A024R5Z3_HUMAN,MSSILPFTPPIVKRLLGWKKGEQNGQEEKWCEKAVKSLVKKLKKTG...,IMMUNE SYSTEM,1
2,tr|A0A0B4Q6D0|A0A0B4Q6D0_9PLVG,MRVKGIRKNYQHLWRGGTLLLGMLMICSAVEKLWVTVYYGVPVWKE...,IMMUNE SYSTEM,1
3,tr|A0A0B4Q7W6|A0A0B4Q7W6_9PLVG,MRVKGIRKNYQHLWRGGTLLLGMLMICSAVEKLWVTVYYGVPVWKE...,IMMUNE SYSTEM,1
4,tr|A0A0B4Q6H6|A0A0B4Q6H6_9PLVG,MRVKGIRKNYQHLWRGGTLLLGMLMICSAVEKLWVTVHYGVPVWKE...,IMMUNE SYSTEM,1


In [4]:
df_2.shape

(84798, 4)

In [5]:
df3 = df_2.sample(frac=0.33)
df3.shape

(27983, 4)

In [6]:
df_hydro = pd.read_csv('uniprot-hydrolase-reviewed.csv')

df_hydro.head()

,name,sequence,classification,type
0,sp|P9WNH5|HSAD_MYCTU,MTATEELTFESTSRFAEVDVDGPLKLHYHEAGVGNDQTVVLLHGGG...,HYDROLASE,0
1,sp|P77044|MHPC_ECOLI,MSYQPQTEAATSRFLNVEEAGKTLRIHFNDCGQGDETVVLLHGSGP...,HYDROLASE,0
2,sp|Q75UV1|NDX1_THETH,MELGAGGVVFNAKREVLLLRDRMGFWVFPKGHPEPGESLEEAAVRE...,HYDROLASE,0
3,sp|Q60928|GGT1_MOUSE,MKNRFLVLGLVAVVLVFVIIGLCIWLPYTSGKPDHVYSRAAVATDA...,HYDROLASE,0
4,sp|P19440|GGT1_HUMAN,MKKKLVVLGLLAVVLVLVIVGLCLWLPSASKEPDNHVYTRAAVAAD...,HYDROLASE,0


In [7]:
df_h2 = df_hydro.sample(frac=0.45)
df_h2.shape

(26386, 4)

In [8]:
df.columns

Index(['classification', 'name', 'sequence', 'structureId', 'type'], dtype='object')

In [9]:
df.shape

(104956, 5)

In [10]:
# concat the dataframes
df_ = pd.concat([df,df3,df_h2])

df_.head()

/Users/scottlew/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


,classification,name,sequence,structureId,type
0,TRANSFERASE,NaN,MEIYEGKLTAEGLRFGIVASRFNHALVDRLVEGAIDCIVRHGGREE...,5MPP,4
1,HYDROLASE,NaN,MKFTLTIAGLLAVGSTAAPTTEKRNPGGIDYVQNYNGDVADFQYNE...,3M4F,0
2,TRANSFERASE,NaN,MRGSHHHHHHGSMKRAVITGLGIVSSIGNNQQEVLASLREGRSGIT...,2BYY,4
3,HYDROLASE,NaN,STGSATTTPIDSLDDAYITPVQIGTPAQTLNLDFDTGSSDLWVFSS...,4YCY,0
4,TRANSFERASE,NaN,GSGMMRYLHKIELELNRLTSRYPFFKKIAFDAEIIKLVDDLNVDEN...,3AQC,4


In [11]:
df_.shape

(159325, 5)

## Remove duplicate protein sequences from data

In [12]:
# remove the duplicate protein sequences
df_ = df_.drop_duplicates(subset='sequence', keep="first")

df_.shape

(157694, 5)

## 5 Classes of Proteins

In [13]:
df_.classification.value_counts()

HYDROLASE        37422
TRANSFERASE      35880
LIGASE           33967
IMMUNE SYSTEM    33437
ISOMERASE        16988
Name: classification, dtype: int64

In [14]:
df_.type.value_counts()

0    37422
4    35880
3    33967
1    33437
2    16988
Name: type, dtype: int64

## Subset the data: use fewer rows of data for quicker comparison of XGBoost with other models

In [15]:
df_new= df_.sample(frac = 1.0)

df_new.shape

(157694, 5)

## Classifier of Protein Sequences: Predict 5 Classes

## CountVectorizer: Count peptide frequency, transform the data

In [16]:
# In this case, peptide frequency is used for analysis
peptide_size = 5
vect_ = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(peptide_size,peptide_size))

## Split the data into training & test sets for classification model

In [17]:
X = vect_.fit_transform(df_new.sequence)
y = df_new.type

In [18]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state =42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(126155, 2818748) (126155,)
(31539, 2818748) (31539,)


In [19]:
y_test.value_counts()

0    7520
4    7200
3    6776
1    6685
2    3358
Name: type, dtype: int64

## Classification Models

## MNB Classifier Model

In [20]:
%%time
mnb = MultinomialNB(alpha=50)
mnb.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("MNB classifier Score: {:.2f}".format(mnb.score(X_test, y_test)))
print()

MNB classifier Score: 0.84

CPU times: user 5.34 s, sys: 1.57 s, total: 6.92 s
Wall time: 7.5 s


In [21]:
%%time
pac = PassiveAggressiveClassifier()
pac.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Passive Aggressive classifier Score: {:.2f}".format(pac.score(X_test, y_test)))
print()

Passive Aggressive classifier Score: 0.93

CPU times: user 1min 1s, sys: 1.87 s, total: 1min 3s
Wall time: 1min 3s


In [22]:
%%time
pac2 = PassiveAggressiveClassifier(loss='squared_hinge')
pac2.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Passive Aggressive classifier with squared hinge loss Score: {:.2f}".format(pac2.score(X_test, y_test)))
print()

Passive Aggressive classifier with squared hinge loss Score: 0.92

CPU times: user 54.2 s, sys: 1.14 s, total: 55.4 s
Wall time: 55.1 s


In [23]:
%%time
sgd = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("SGD classifier Score: {:.2f}".format(sgd.score(X_test, y_test)))
print()

SGD classifier Score: 0.92

CPU times: user 1min 25s, sys: 2.57 s, total: 1min 27s
Wall time: 1min 30s


In [24]:
%%time
sgd2 = linear_model.SGDClassifier(loss= 'log',max_iter=1000, tol=1e-3)
sgd2.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("SGD classifier Score: {:.2f}".format(sgd2.score(X_test, y_test)))
print()

SGD classifier Score: 0.92

CPU times: user 1min 18s, sys: 2.42 s, total: 1min 20s
Wall time: 1min 20s


In [25]:
%%time
per = Perceptron(tol=1e-3, random_state=0).fit(X_train,y_train)
# evaluate accuracy of our model on test data
print("Perceptron classifier Score: {:.2f}".format(per.score(X_test, y_test)))
print()

Perceptron classifier Score: 0.92

CPU times: user 52.1 s, sys: 1.27 s, total: 53.3 s
Wall time: 53.2 s


In [26]:
# Generate Confusion Matrix for Perceptron Model
actual = y_test
predictions = per.predict(X_test)
print('Confusion Matrix for Perceptron Model')
print()
cm = confusion_matrix(actual,predictions)
print(cm)

Confusion Matrix for Perceptron Model

[[6625   85   90  203  517]
 [  98 6482   29   29   47]
 [  68    7 3175   37   71]
 [ 185   19   38 6340  194]
 [ 463   68   85  177 6407]]


## Voting Models

In [27]:
%%time
clf1 = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2 = PassiveAggressiveClassifier(C=0.1)
#clf3 = MultinomialNB(alpha=50)

eclf1 = VotingClassifier(estimators=[('sgd',clf1), ('pac',clf2)], voting='hard')
eclf1.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf1.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.92

CPU times: user 2min 8s, sys: 2.96 s, total: 2min 11s
Wall time: 2min 13s


In [28]:
# Generate Confusion Matrix for Voting Model
actual = y_test
predictions = eclf1.predict(X_test)
print('Confusion Matrix for Voting Model')
print()
cm = confusion_matrix(actual,predictions)
print(cm)

Confusion Matrix for Voting Model

[[7051   24   17  109  319]
 [ 199 6458    0    9   19]
 [ 211    9 3050   18   70]
 [ 424   26    7 6189  130]
 [ 691   21    9  116 6363]]


In [29]:
%%time
clf1a = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2a = PassiveAggressiveClassifier(C=0.1)
clf3a = MultinomialNB(alpha=50)

eclf2 = VotingClassifier(estimators=[('sgd', clf1a), ('pac', clf2a), ('mnb', clf3a)], voting='hard')
eclf2.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf2.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.93

CPU times: user 2min 8s, sys: 2.33 s, total: 2min 10s
Wall time: 2min 8s


In [30]:
# Generate Confusion Matrix for Voting Model
actual = y_test
predictions = eclf2.predict(X_test)
print('Confusion Matrix for Voting Model')
print()
cm = confusion_matrix(actual,predictions)
print(cm)

Confusion Matrix for Voting Model

[[6942   30   14  158  376]
 [ 179 6467    0   15   24]
 [ 195   14 3040   27   82]
 [ 316   17    4 6296  143]
 [ 589   19    5  121 6466]]


In [31]:
%%time
clf1b = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2b = PassiveAggressiveClassifier(C=0.1)
clf3b = linear_model.SGDClassifier(loss= 'log',max_iter=1000, tol=1e-3)

eclf3 = VotingClassifier(estimators=[('sgd', clf1b), ('pac', clf2b), ('sgd_lg', clf3b)], voting='hard')
eclf3.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf3.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.93

CPU times: user 3min 20s, sys: 2.91 s, total: 3min 23s
Wall time: 3min 20s


In [32]:
%%time
clf1c = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2c = PassiveAggressiveClassifier(C=0.1)
clf3c = Perceptron(tol=1e-3, random_state=0)

eclf4 = VotingClassifier(estimators=[('sgd', clf1c), ('pac', clf2c), ('per', clf3c)], voting='hard')
eclf4.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf4.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.93

CPU times: user 2min 36s, sys: 2.91 s, total: 2min 39s
Wall time: 2min 36s


In [33]:
vote_prediction4 = eclf4.predict(X_test)
print(classification_report(vote_prediction4, y_test))

              precision    recall  f1-score   support

           0       0.93      0.85      0.89      8195
           1       0.97      0.99      0.98      6552
           2       0.93      0.99      0.96      3155
           3       0.92      0.97      0.94      6459
           4       0.90      0.91      0.91      7178

    accuracy                           0.93     31539
   macro avg       0.93      0.94      0.94     31539
weighted avg       0.93      0.93      0.93     31539



In [34]:
%%time
clf1d = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2d = PassiveAggressiveClassifier(C=0.1)
clf3d = Perceptron(tol=1e-3, random_state=0)
clf4d = MultinomialNB(alpha=50)

eclf5 = VotingClassifier(estimators=[('sgd', clf1d), ('pac', clf2d), ('per', clf3d),('mnb',clf4d)], voting='hard')
eclf5.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf5.score(X_test, y_test)))
print()

Majority Voting Classifier Score: 0.93

CPU times: user 3min 2s, sys: 6.77 s, total: 3min 9s
Wall time: 13min 44s


In [35]:
vote_prediction5 = eclf5.predict(X_test)
print(classification_report(vote_prediction5, y_test))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89      8126
           1       0.97      0.99      0.98      6527
           2       0.93      0.99      0.96      3138
           3       0.93      0.95      0.94      6690
           4       0.90      0.91      0.90      7058

    accuracy                           0.93     31539
   macro avg       0.93      0.94      0.93     31539
weighted avg       0.93      0.93      0.93     31539



In [36]:
%%time
clf1e = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2e = PassiveAggressiveClassifier(C=0.1)
clf3e = Perceptron(tol=1e-3, random_state=0)
clf4e = MultinomialNB(alpha=50)
clf5e = LogisticRegression(C=3.0)

eclf_6 = VotingClassifier(estimators=[('sgd', clf1e), ('pac', clf2e), \
                                      ('per', clf3e),('mnb',clf4e),('lr',clf5e)], voting='hard')
eclf_6.fit(X_train, y_train)
# evaluate accuracy of our model on test data
print("Majority Voting Classifier Score: {:.2f}".format(eclf_6.score(X_test, y_test)))
print()

/Users/scottlew/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/scottlew/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/scottlew/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Majority Voting Classifier Score: 0.93

CPU times: user 57min 22s, sys: 32.8 s, total: 57min 55s
Wall time: 1h 5min 12s


In [37]:
vote_prediction_6 = eclf_6.predict(X_test)
print(classification_report(vote_prediction_6, y_test))

              precision    recall  f1-score   support

           0       0.92      0.87      0.89      7996
           1       0.97      0.99      0.98      6520
           2       0.93      0.99      0.96      3134
           3       0.93      0.96      0.94      6599
           4       0.91      0.90      0.91      7290

    accuracy                           0.93     31539
   macro avg       0.93      0.94      0.94     31539
weighted avg       0.93      0.93      0.93     31539



## Cross Validation of Models

In [38]:
clf1e = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2e = PassiveAggressiveClassifier(C=0.1)

eclf_cv = VotingClassifier(estimators=[('sgd',clf1e), ('pac',clf2e)], voting='hard')
scores = cross_val_score(eclf_cv,X,y, cv = 5)
print("Cross-validation scores for Voting Classifier: {}".format(scores))
print()
print("The average accuracy score for Voting Classifier is: ")
print(np.mean(scores))

Cross-validation scores for Voting Classifier: [ 0.92010399  0.91763102  0.9188915   0.91936456  0.91990361]

The average accuracy score for Voting Classifier is: 
0.919178933751


In [39]:
clf1f = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2f = PassiveAggressiveClassifier(C=0.1)
clf3f = Perceptron(tol=1e-3, random_state=0)

eclf_cv2 = VotingClassifier(estimators=[('sgd',clf1f), ('pac',clf2f),('per', clf3f)], voting='hard')
scores = cross_val_score(eclf_cv2,X,y, cv = 5)

print("Cross-validation scores for Voting Classifier: {}".format(scores))
print()
print("The average accuracy score for Voting Classifier is: ")
print(np.mean(scores))

Cross-validation scores for Voting Classifier: [ 0.92923496  0.93107384  0.92881603  0.92862352  0.92878207]

The average accuracy score for Voting Classifier is: 
0.929306084591


In [40]:
clf1g = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf2g = PassiveAggressiveClassifier(C=0.1)
clf3g = Perceptron(tol=1e-3, random_state=0)
clf4g = MultinomialNB(alpha=50)

eclf_cv3 = VotingClassifier(estimators=[('sgd',clf1g), ('pac',clf2g),('per', clf3g),('mnb',clf4g)], voting='hard')
scores3 = cross_val_score(eclf_cv3,X,y, cv = 5)

print("Cross-validation scores for Voting Classifier: {}".format(scores3))
print()
print("The average accuracy score for Voting Classifier is: ")
print(np.mean(scores3))

Cross-validation scores for Voting Classifier: [ 0.92841064  0.93050315  0.9274843   0.92903574  0.92824302]

The average accuracy score for Voting Classifier is: 
0.928735370267
